<a href="https://colab.research.google.com/github/Devarsh-19/Vegetable-classification/blob/main/project_veg_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!pip install -U matplotlib

     |████████████████████████████████| 11.2 MB 25.1 MB/s 
     |████████████████████████████████| 929 kB 44.2 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import os # file directory and file sysstem ops
import numpy as np # for nd arrays

import torch  #for deep learning
from torch import nn

from PIL import Image
import cv2
from matplotlib import pyplot as plt

In [ ]:
import torchvision
from torchvision import transforms, datasets

In [ ]:
from tqdm import tqdm #progess report and loop opertarions

In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download misrakahmed/vegetable-image-dataset

 99% 529M/534M [00:02<00:00, 256MB/s]
100% 534M/534M [00:02<00:00, 225MB/s]


In [ ]:
!unzip -qq vegetable-image-dataset.zip

In [ ]:
! mv Vegtable\ Images Vegetable_Images 

mv: cannot stat 'Vegtable Images': No such file or directory


#EDA
--------------------
STEP 1 : Find number of classes and number of images in each class  
STEP 2 : Size Analysis - heights, widths, resolutions, aspect rations  
STEP 3 : Look for blurred images  
STEP 4 : Look for low contrast images  
STEP 5 : Look for noisy images  
STEP 6 : Look for bright and dark images  


In [ ]:
train_vegetables = datasets.ImageFolder("Vegetable Images/train",transform = None)
val_vegetables = datasets.ImageFolder("Vegetable Images/validation",transform = None)
test_vegetables = datasets.ImageFolder("Vegetable Images/test",transform = None)

In [ ]:
print(train_vegetables), print(val_vegetables), print(test_vegetables)

Dataset ImageFolder
    Number of datapoints: 15000
    Root location: Vegetable Images/train
Dataset ImageFolder
    Number of datapoints: 3000
    Root location: Vegetable Images/validation
Dataset ImageFolder
    Number of datapoints: 3000
    Root location: Vegetable Images/test


(None, None, None)

In [ ]:
vegetable_classes = train_vegetables.classes
print(f"Classes: {vegetable_classes}")

Classes: ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']


In [ ]:
print("No. of classes: {}".format(len(vegetable_classes)))

No. of classes: 15


In [ ]:
targets = train_vegetables.targets

In [ ]:
class_strength = np.unique(targets, return_counts= True)

In [ ]:
class_strength

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000]))

EDA: STEP 2

In [ ]:
widths = []
heights = []
resolutions = []
aspect_ratios = []
imgs = []

for i in tqdm(range(len(train_vegetables.imgs))):
  img_path, label = train_vegetables.imgs[i]
  img = Image.open(img_path)
  w, h = img.size

  imgs.append(img_path)
  heights.append(h)
  widths.append(w)
  resolutions.append(h*w)
  aspect_ratios.append(w/h)

100%|██████████| 15000/15000 [00:04<00:00, 3141.58it/s]


In [ ]:
print(len(heights))
print(min(heights), max(heights))

15000
193 224


The standard resolution used to train the neural network images is 128 by 128.   
for 1st image 128 x 128  
for 2nd 224 x 224  
for 3rd 256 x 256 and so on  
make sure to keep resolution below 400 x 400  

IMAGE AUGMENTATION -> maipulate image data

In [ ]:
transform = transforms.Compose([
                                transforms.Resize((224,224)),
                                transforms.ToTensor()
])

#on colab

train_vegetables = datasets.ImageFolder("Vegetable Images/train",transform = transform)
val_vegetables = datasets.ImageFolder("Vegetable Images/validation",transform = transform)
test_vegetables = datasets.ImageFolder("Vegetable Images/test",transform = transform)

In [ ]:
print(train_vegetables), print(val_vegetables), print(test_vegetables)

Dataset ImageFolder
    Number of datapoints: 15000
    Root location: Vegetable Images/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 3000
    Root location: Vegetable Images/validation
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 3000
    Root location: Vegetable Images/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )


(None, None, None)

### DATA SAMPLER AND RANDOM SAMPLING  
CUDA - Compute Unified Device Architecture
Validataion Data - Ground Truth

In [ ]:
device = torch.device("cuda:0")

In [ ]:
train_loader = torch.utils.data.DataLoader(train_vegetables,
                                      batch_size = 32,
                                      shuffle = True,
                                      pin_memory = True)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_vegetables,
                                      batch_size = 45,
                                      shuffle = False,
                                      pin_memory = True)

In [ ]:
val_loader = torch.utils.data.DataLoader(val_vegetables,
                                      batch_size = 40,
                                      shuffle = False,
                                      pin_memory = True)

In [ ]:
print(len(train_loader)), print(len(test_loader)), print(len(val_loader))

469
67
75


(None, None, None)